# Mezcla binaria (NP's y Mesógenos)

In [2]:
#-----Importa los paquetenes necesartios para correr la simulación

from __future__ import division
import hoomd
import hoomd.md

#-----Define algunas variables relevantes

t_max = 5.0;
p_0   = 0.5; 
p_1   = 0.7;
steps_ramp = 1e5;

#-----Crea un contexto de simulación

hoomd.context.initialize("--mode=cpu", memory_traceback = True);

#-----Extrae la configuración del centro de masa de las partículas

system = hoomd.init.read_gsd("T_CM&NP_5.0_P_0.5_ramp.gsd", frame = -1);

#-----Define cada mesógeno en el marco de referencia local del cuerpo.

rigid = hoomd.md.constrain.rigid();
rigid.set_param('M', 
                types = ['A']*8,
                positions = [(-4,0,0),(-3,0,0),(-2,0,0),(-1,0,0),
                           (1,0,0),(2,0,0),(3,0,0),(4,0,0)]);

#-----Crea los mesógenos.

rigid.create_bodies();

#-----Define la energía potencial.

nl = hoomd.md.nlist.tree();

lj = hoomd.md.pair.lj(r_cut = 3.5, nlist = nl)
lj.set_params(mode = 'shift')

#-----Declara las interacciones entre elementos de la mezcla.

lj.pair_coeff.set('NP','NP', epsilon = 1.0, sigma = 5.0);
lj.pair_coeff.set('M' ,'M' , epsilon = 1.0, sigma = 1.0);
lj.pair_coeff.set('A' ,'A' , epsilon = 1.0, sigma = 1.0);
lj.pair_coeff.set('M' ,'A' , epsilon = 1.0, sigma = 1.0);
lj.pair_coeff.set('NP','M' , epsilon = 1.0, sigma = 3.0);
lj.pair_coeff.set('NP','A' , epsilon = 1.0, sigma = 3.0);

#-----Selecciona un integrador estándar.

hoomd.md.integrate.mode_standard(dt = 0.005);

#------Define dos grupos y efectua su unión.

nanoparticles = hoomd.group.type(name = 'Nano_Particles', type = 'NP');
mesogens = hoomd.group.rigid_center();
groupNP_mes = hoomd.group.union(name = "NP_Mes", a = nanoparticles, b = mesogens);

#----Integra usando un NPT.

press = hoomd.variant.linear_interp(points = [(0,p_0), (steps_ramp, p_1)]);

npt = hoomd.md.integrate.npt(group = groupNP_mes, kT = t_max, tau = 0.5, tauP = 1.0, P = press);
npt.randomize_velocities(seed=42)

#-----Guarda los datos en archivos .log y .gsd

log_file = "T_" + str(t_max) + "_P_" + str(p_1) + "_ramp.log"
gsd_file = "T_" + str(t_max) + "_P_" + str(p_1) + "_ramp.gsd" 
meso_gsd_file = "T_CM&NP_" + str(t_max) + "_P_" + str(p_1) + "_ramp.gsd"

log = hoomd.analyze.log(filename = log_file,
                         quantities = ['num_particles',
                                     'ndof',
                                     'translational_ndof',
                                     'rotational_ndof',
                                     'potential_energy',
                                     'kinetic_energy',
                                     'translational_kinetic_energy',
                                     'rotational_kinetic_energy',
                                     'temperature',
                                     'pressure',
                                      'volume'],
                         period = 1e3,
                         overwrite = True);
gsd = hoomd.dump.gsd(gsd_file,
               period = 1e3,
               group = hoomd.group.all(),
               overwrite = True); 
meso_gsd = hoomd.dump.gsd(meso_gsd_file,
               period = 1e3,
               group = groupNP_mes,
               overwrite = True); 

notice(2): Group "all" created containing 1004 particles
notice(2): constrain.rigid(): Creating 1000 rigid bodies (adding 8000 particles)
-----
You are using tree neighbor lists. Please cite the following:
* M P Howard, J A Anderson, A Nikoubashman, S C Glotzer, and A Z
  Panagiotopoulos. "Efficient neighbor list calculation for molecular simulation
  of colloidal systems using graphics processing units", Computer Physics
  Communications 203 (2016) 45--52
-----
notice(2): Group "Nano_Particles" created containing 4 particles
notice(2): Group "rigid_center" created containing 1000 particles
notice(2): Group "NP_Mes" created containing 1004 particles


In [3]:
hoomd.run(steps_ramp)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 9004
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: yes
** starting run **
Time 00:00:10 | Step 99270 / 199000 | TPS 26.9847 | ETA 01:01:35
Time 00:00:20 | Step 99543 / 199000 | TPS 27.2398 | ETA 01:00:51
Time 00:00:30 | Step 99797 / 199000 | TPS 25.2453 | ETA 01:05:29
Time 00:00:40 | Step 100060 / 199000 | TPS 26.286 | ETA 01:02:43
Time 00:00:50 | Step 100313 / 199000 | TPS 25.1594 | ETA 01:05:22
Time 00:01:00 | Step 100580 / 199000 | TPS 26.6079 | ETA 01:01:38
Time 00:01:10 | Step 100858 / 199000 | TPS 27.6214 | ETA 00:59:13
Time 00:01:20 | Step 101137 / 199000 | TPS 27.8778 | ETA 00:58:30
Time 00:01:30 | Step 101415 / 199000 | TPS 27.7672 | ETA 00:58:34
Time 00:01:40 | Step 101699 / 199000 | TPS 28.3451 | ETA 00:57:12
Time 00:01:50 | Step 101970 / 199000 | TPS 26.9725 | ETA 00:59:57
Time 00:02:00 | Step 102240 / 19900

In [4]:
system.get_metadata()

OrderedDict([('box', <hoomd.data.boxdim at 0x10b694a90>),
             ('particles', <hoomd.data.particle_data at 0x11a75e4a8>),
             ('number_density', 0.41075409255245354),
             ('bonds', <hoomd.data.bond_data at 0x11a75e550>),
             ('angles', <hoomd.data.angle_data at 0x11a75e5c0>),
             ('dihedrals', <hoomd.data.dihedral_data at 0x11a75e630>),
             ('impropers', <hoomd.data.dihedral_data at 0x11a75e6a0>),
             ('constraints', <hoomd.data.constraint_data at 0x11a75e710>),
             ('pairs', <hoomd.data.bond_data at 0x11a75e780>),
             ('timestep', 199000)])